In [2]:
import numpy as np
import scipy as sp
from tqdm import tqdm
A = np.array([[1.0,1.0,-1.0],[3.0,-1.0,1.0],[1.0, -3.0,2.0]])
b = np.array([[0],[4],[1.1]],dtype=float)
A

array([[ 1.,  1., -1.],
       [ 3., -1.,  1.],
       [ 1., -3.,  2.]])

### Cramers Rule

In [3]:
def Cramers_Method(A,b):
    det = np.linalg.det(A)
    b = b.reshape(-1)  # Flatten b to 1D array
    dets = np.zeros(b.size)
    for i in range(b.size):
        temp = A.copy()
        temp[:,i] = b  # Replace column i instead of row i
        dets[i] = np.linalg.det(temp)
    dets = dets/det
    return dets
Cramers_Method(A,b)

array([1. , 1.9, 2.9])

In [4]:
def Inverse_Method(A,b):
    A_inv = np.linalg.inv(A)
    return np.matmul(A_inv,b)
Inverse_Method(A,b)

array([[1. ],
       [1.9],
       [2.9]])

In [5]:
C = np.array([[1,1],[1.00001,1]],dtype=float)
d = np.array([2,2.0],dtype=float)

In [6]:
def Condition_Number(A):
    if A.shape[0]!=A.shape[1]:
        raise Exception('A is not a square matrix')
    A_inv = np.linalg.inv(A)
    norm_A = -100000
    n = A.shape[0]
    for i in range(0,n):
        sum = 0
        for j in range(0,n):
            sum+= np.abs(A[i,j])
        if sum>norm_A:
            norm_A = sum
    norm_Ainv = -100000
    n = A.shape[0]
    for i in range(0,n):
        sum = 0
        for j in range(0,n):
            sum+= np.abs(A_inv[i,j])
        if sum>norm_Ainv:
            norm_Ainv = sum
    return norm_A*norm_Ainv

Condition_Number(A)

np.float64(24.0)

In [7]:
def GaussElimination_Pivoting(A,b):

    if A.shape[0]!=A.shape[1]:
        raise ValueError('Error The matrix is not a square matrix')
    elif A.shape[0]!=b.shape[0]:
        raise ValueError('Error A and b are not compatible')
    #Forming upper diagonal matrix
    for k in range(0,A.shape[0]-1):
        idx = np.argmax(np.abs(A[k:, k])) + k
        if idx != k:
            A[[k, idx]] = A[[idx, k]]
            b[[k, idx]] = b[[idx, k]]
        for i in range(k+1,A.shape[0]):
            if A[k,k] == 0:
                raise ZeroDivisionError('Division by zero')
            m = A[i,k]/A[k,k]
            A[i,:] = A[i,:] - m*A[k,:]
            b[i] = b[i] - m*b[k]
    x = np.zeros_like(b)

    #Backward substitution
    for i in (range(b.shape[0]-1,-1,-1)):
        x[i] = (b[i] - np.dot(A[i,i+1:A.shape[0]],x[i+1:b.shape[0]]))/A[i,i]
    return x


x = GaussElimination_Pivoting(A,b)
x

array([[1. ],
       [1.9],
       [2.9]])

In [8]:
P,L,U = sp.linalg.lu(A)

P

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [9]:
Q,R = sp.linalg.qr(A)
Q
R

array([[ 3.        , -1.        ,  1.        ],
       [ 0.        , -2.66666667,  1.66666667],
       [ 0.        ,  0.        , -0.5       ]])

### Gauss Jacobi

In [10]:
A = np.array([[10,-1,2,0],[-1,11,-1,3],[2,-1,10,-1],[0,-3,-1,8]])
b = np.array([[6,25,-11,15]])
b = np.transpose(b)
A

array([[10, -1,  2,  0],
       [-1, 11, -1,  3],
       [ 2, -1, 10, -1],
       [ 0, -3, -1,  8]])

In [11]:
n = np.size(b)
n

4

In [12]:
D = np.diagonal(A)
D = D.reshape((n,1))

In [13]:
z = np.diag(D)
z

array([10])

In [14]:
LU = A - z

In [15]:
xg = np.ones_like(b)
xg

array([[1],
       [1],
       [1],
       [1]])

In [16]:
x = (b - np.dot(LU,xg))/D
x

array([[3.5       ],
       [4.81818182],
       [1.9       ],
       [6.375     ]])

In [17]:
res = x-xg
tol=np.linalg.norm(res)
tol

np.float64(7.108385006222877)

In [18]:
def GaussJacobi(A,b,e = 1e-6,max_iter = 1000):
    n = A.shape[0]
    b = np.reshape(b,[n,1])
    D = np.reshape(np.diagonal(A),[n,1])
    B = A - np.diag(np.diag(A))
    xg = np.reshape(np.ones_like(b),[n,1])
    for it in tqdm(range(max_iter)):
        x = (b-np.matmul(B,xg))/D
        tol = np.linalg.norm(x-xg,ord=2)
        if tol<e:
            print(f'Iterations : {it}')
            break
        xg = x.copy()
    return x

GaussJacobi(A,b)

  1%|          | 12/1000 [00:00<00:00, 1497.16it/s]

Iterations : 12


array([[ 0.94049585],
       [ 1.62975218],
       [-0.88760334],
       [ 2.37520654]])

In [19]:
def GaussSeidel(A,b,e = 1e-6,maxiter =1000):
    n = A.shape[0]
    x = np.ones_like(b,dtype=np.float64)
    it = 0
    tol = np.linalg.norm(b-np.dot(A,x),ord=2)
    while(tol>e and it<maxiter):
        for i in range(n):
            sigma = 0
            for j in range(n):
                if i!=j:
                    sigma+= A[i,j]*x[j]
            x[i] = (b[i] - sigma)/A[i][i]
        tol = np.linalg.norm(b-np.dot(A,x),ord=2)
        it+=1
    if it==maxiter:
        raise Exception("Could not converge to a solution")
    return x

GaussSeidel(A,b)

array([[ 0.94049591],
       [ 1.62975203],
       [-0.8876033 ],
       [ 2.3752066 ]])

In [ ]:
A = np.array([[1.0,1.0,-1.0],[3.0,-1.0,1.0],[1.0, -3.0,2.0]])
b = np.array([[0.0],[4.0],[1.1]],dtype=float)

In [37]:
def LU_decomposition(A):
    n = A.shape[0]
    L = np.zeros_like(A)
    U = np.zeros((n,n),dtype=float)
    for i in range(n):
        for j in range(n):
            if i<=j:
                U[i,j] = A[i,j]
                for k in range(i):
                    U[i,j] -= L[i,k]*U[k,j]
                if i==j:
                    L[i,j] = 1
                else :
                    L[i,j] = 0
            else :
                L[i,j] = A[i,j]
                for k in range(j):
                    L[i,j] -=L[i,k]*U[k,j]
                L[i,j]/=U[j,j]
                U[i,j] = 0
            
    return L, U

def forward_substitution(L, b):
    n = L.shape[0]
    y = np.zeros_like(b, dtype=float)
    y[0] = b[0]/L[0,0]
    for i in range(1,n):
        y[i] = (b[i] - np.dot(L[i,:i], y[:i]))/L[i,i]
    return y

def backward_substitution(U, y):
    n = U.shape[0]
    x = np.zeros_like(y, dtype=float)
    x[-1] = y[-1]/U[-1,-1]
    for i in range(n-2,-1,-1):
        x[i] = (y[i] - np.dot(U[i,i+1:], x[i+1:]))/U[i,i]
    return x

L,U = LU_decomposition(A)

y = forward_substitution(L,b)
x = backward_substitution(U,y)
x

array([[1. ],
       [1.9],
       [2.9]])